In [ ]:
import sys
import os
import rasterio
import rasterio.warp


# Import raster helpers
sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")

import GOSTRocks.dataMisc as dataMisc

# Import GOST urban functions
sys.path.append("../../../src")
import GOST_Urban.urban_helper as helper

%load_ext autoreload
%autoreload 2

In [ ]:
pop_urban_folder = (
    "/home/wb411133/data/Projects/MR_Novel_Urbanization/Data/COG_URBAN_DATA_new_naming"
)
in_bounds = dataMisc.get_geoboundaries("COG", "ADM1")

out_folder = "/home/wb411133/temp"

In [ ]:
# summarize worldpop urbanization and population
class urban_summary:
    def __init__(self, f_base, in_folder):
        self.f_base = f_base
        self.in_folder = in_folder

        self.pop_layer = os.path.join(in_folder, f"{f_base}.tif")
        self.hd_layer = os.path.join(in_folder, f"{f_base}_urban_hd.tif")
        self.urb_layer = os.path.join(in_folder, f"{f_base}_urban.tif")

    def compare_populations(self, boundaries):
        calculator = helper.summarize_population(
            self.pop_layer, boundaries, self.urb_layer, self.hd_layer
        )
        pop_res = calculator.calculate_zonal()
        return pop_res

In [ ]:
source_folder = os.path.join(pop_urban_folder, "FINAL_STANDARD")
comp1_base = "cog_cpo20"
comp2_base = "cog_cpo20_WB"

calculator = urban_summary(comp1_base, source_folder)
cpo20_pop_res = calculator.compare_populations(in_bounds)

calculator = urban_summary(comp2_base, source_folder)
cpo20WB_pop_res = calculator.compare_populations(in_bounds)

combo_res = cpo20_pop_res.loc[:, [x for x in cpo20_pop_res.columns if "SUM" in x]].join(
    cpo20WB_pop_res.loc[:, [x for x in cpo20WB_pop_res.columns if "SUM" in x]]
)
combo_res.to_csv(os.path.join(out_folder, "COG_250m_urban_population_summary.csv"))

In [ ]:
cpo20WB_pop_res

In [ ]:
source_folder = os.path.join(pop_urban_folder, "FINAL_STANDARD_1KM")
comp1_base = "cog1k_cpo20"
comp2_base = "cog1k_cpo20_WB"

calculator = urban_summary(comp1_base, source_folder)
cpo20_pop_res = calculator.compare_populations(in_bounds)

calculator = urban_summary(comp2_base, source_folder)
cpo20WB_pop_res = calculator.compare_populations(in_bounds)

combo_res = cpo20_pop_res.loc[:, [x for x in cpo20_pop_res.columns if "SUM" in x]].join(
    cpo20WB_pop_res.loc[:, [x for x in cpo20WB_pop_res.columns if "SUM" in x]]
)
combo_res.to_csv(os.path.join(out_folder, "COG_1km_urban_population_summary.csv"))

# Reasonable-ness test

In [ ]:
source_folder = os.path.join(pop_urban_folder, "FINAL_STANDARD")
comp1_base = "cog_cpo20"
comp2_base = "cog_cpo20_WB"

com1_pop = rasterio.open(os.path.join(source_folder, f"{comp1_base}.tif")).read()
com2_pop = rasterio.open(os.path.join(source_folder, f"{comp2_base}.tif")).read()

In [ ]:
com1_pop[com1_pop > 0].sum()

In [ ]:
com2_pop[com1_pop > 0].sum()